In [37]:
import pandas as pd
import numpy as np

In [38]:
pd.set_option('display.max_columns', None)

In [39]:
df = pd.read_csv('hr_raw_data.csv', index_col= 0)

In [40]:
#AÑADIR ($) A NOMBRE DE LAS COLUMNAS: 
columnas_dolar = ["dailyrate", "hourlyrate", "monthlyincome", "monthlyrate", "sameasmonthlyincome", "salary"]
for c in columnas_dolar:
    df.rename(columns={c: c + "($)"}, inplace=True)



In [41]:
#LIMPIEZA DE DATOS PARA CONVENTIR DE OBJECT A FLOAT
columnas_a_float = ["monthlyincome($)", "totalworkingyears", "salary($)", "monthlyrate($)", "sameasmonthlyincome($)"]

for columna in columnas_a_float:

    df[columna] = df[columna].astype(str).str.replace('$', '', regex=False)
    print(f"La columna '{columna}' fue procesada para eliminación de símbolo $.")

    df[columna] = df[columna].str.replace(',', '.')
    print(f"La columna '{columna}' fue procesada para cambio de coma.")
    
    df[columna] = pd.to_numeric(df[columna], errors="coerce")
    print(f"La columna '{columna}' fue procesada para float.")


La columna 'monthlyincome($)' fue procesada para eliminación de símbolo $.
La columna 'monthlyincome($)' fue procesada para cambio de coma.
La columna 'monthlyincome($)' fue procesada para float.
La columna 'totalworkingyears' fue procesada para eliminación de símbolo $.
La columna 'totalworkingyears' fue procesada para cambio de coma.
La columna 'totalworkingyears' fue procesada para float.
La columna 'salary($)' fue procesada para eliminación de símbolo $.
La columna 'salary($)' fue procesada para cambio de coma.
La columna 'salary($)' fue procesada para float.
La columna 'monthlyrate($)' fue procesada para eliminación de símbolo $.
La columna 'monthlyrate($)' fue procesada para cambio de coma.
La columna 'monthlyrate($)' fue procesada para float.
La columna 'sameasmonthlyincome($)' fue procesada para eliminación de símbolo $.
La columna 'sameasmonthlyincome($)' fue procesada para cambio de coma.
La columna 'sameasmonthlyincome($)' fue procesada para float.


In [42]:
#UNIFICAR Y LIMPIAR DATOS EN REMOTEWORK(BOOLEANO)
df["remotework"] = df["remotework"].replace({"Yes": True,
                                             "1": True,
                                             "0": False,
                                             "True": True,
                                             "False": False})

/var/folders/fb/137nxmdd1m7bk6nfnkpl5j5r0000gn/T/ipykernel_69733/3107660277.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["remotework"] = df["remotework"].replace({"Yes": True,


In [43]:
#PONER TODOS LOS ELEMENTOS DE  LAS COLUMNAS EN MINÚSCULA
columnas_de_texto = df.select_dtypes(include=['object']).columns 

for columna in columnas_de_texto:
    df[columna] = df[columna].str.lower()
    

In [44]:
#ELIMINAR LOS GUIONES BAJOS DE LOS VALORES DE LAS COLUMNAS
for columna in columnas_de_texto:
    df[columna] = df[columna].str.replace('_', ' ', regex=False)
    df[columna] = df[columna].str.replace('-', ' ', regex=False)

In [45]:
#NORMALIZACIÓN MARITALSTATUS. 
df['maritalstatus'] = df['maritalstatus'].fillna('unknown').replace('marreid', 'married')


#DISTANCEFROMHOME
numerador = df[df['distancefromhome']<0].shape[0]
denominador = df.shape[0] #ACCEDEMOS AL NÚMERO TOTAL DE FILAS
porcentaje_negativos = (numerador/denominador)*100 #EVITAMOS NÚMEROS MÁGICOS
print(f'porcentaje de negativos que contiene distancefromhome: {porcentaje_negativos}')
df['distancefromhome'] = abs(df['distancefromhome']) #CON ABS TODA LA SERIE FUNCIONA COMO UN VALOR ABSOLUTO
print(f'comprobamos cantidad de negativos en distancefromhome, tras conversion en valor absoluto: {df[df['distancefromhome']<0].shape[0]}')


porcentaje de negativos que contiene distancefromhome: 12.038140643623361
comprobamos cantidad de negativos en distancefromhome, tras conversion en valor absoluto: 0


In [46]:
#NORMALIZACION DE AGE
mapeo = {'forty seven':47,'fifty eight':58, 'thirty six':36, 'fifty five':55,
       'fifty two':52, 'thirty one':31, 'thirty':30, 'twenty six':26, 'thirty seven':37,
       'thirty two':32, 'twenty four':24}


In [47]:
df["age"] = df["age"].replace(mapeo)

In [48]:
#CONVERSION DE OBJECT A INT
columnas_a_convertir = ["age","worklifebalance", "performancerating", "numberchildren", "yearsincurrentrole"]
for i in df.index:
    for col in columnas_a_convertir:
        # valor original
        valor_original = df.loc[i, col]
        # reemplazar coma por punto
        valor = str(valor_original).replace(",", ".")
        try:
            # Intentar convertir a número
            numero = float(valor)
            df.loc[i, col] = int(numero)
        except:
            # dejar el valor tal cual
            df.loc[i, col] = valor_original
#  Convertir cada columna a entero nullable (Int64)
for col in columnas_a_convertir:
    df[col] = df[col].astype("Int64")

In [49]:
df.head()

,age,attrition,businesstravel,dailyrate($),department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate($),jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome($),monthlyrate($),numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome($),datebirth,salary($),roledepartament,numberchildren,remotework
0,51,no,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,research director,3,unknown,16280.83,42330.17,7,y,no,13,3,3,full time,0,NaN,5,3,20,<NA>,15,15,16280.83,1972,195370.0,NaN,<NA>,True
1,52,no,NaN,2063.388889,NaN,1,4,life sciences,1,2,3,0,NaN,2,5,manager,3,unknown,NaN,43331.17,0,NaN,NaN,14,3,1,NaN,1,34.0,5,3,33,<NA>,11,9,NaN,1971,199990.0,NaN,<NA>,True
2,42,no,travel rarely,1984.253968,research & development,4,2,technical degree,1,3,3,0,NaN,3,5,manager,4,married,NaN,41669.33,1,NaN,no,11,3,4,NaN,0,22.0,3,<NA>,22,<NA>,11,15,NaN,1981,192320.0,manager research & development,<NA>,True
3,47,no,travel rarely,1771.404762,NaN,2,4,medical,1,4,1,1,NaN,3,4,research director,3,married,14307.50,37199.50,3,y,NaN,19,3,2,full time,2,NaN,2,<NA>,20,<NA>,5,6,14307.50,1976,171690.0,NaN,<NA>,False
4,46,no,NaN,1582.771346,NaN,3,3,technical degree,1,5,1,1,NaN,4,4,sales executive,1,divorced,12783.92,33238.20,2,y,no,12,3,4,NaN,1,NaN,5,3,19,<NA>,2,8,12783.92,1977,NaN,NaN,<NA>,False


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1678 entries, 0 to 1677
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       1678 non-null   Int64  
 1   attrition                 1678 non-null   object 
 2   businesstravel            877 non-null    object 
 3   dailyrate($)              1678 non-null   float64
 4   department                312 non-null    object 
 5   distancefromhome          1678 non-null   int64  
 6   education                 1678 non-null   int64  
 7   educationfield            904 non-null    object 
 8   employeecount             1678 non-null   int64  
 9   employeenumber            1678 non-null   int64  
 10  environmentsatisfaction   1678 non-null   int64  
 11  gender                    1678 non-null   int64  
 12  hourlyrate($)             411 non-null    float64
 13  jobinvolvement            1678 non-null   int64  
 14  joblevel     

## LIMPIEZA DE NULOS

In [51]:
#LIMPIEZA HOURLYRATE

print("Valores nulos ANTES de la corrección de hourlyrate:", df['hourlyrate($)'].isnull().sum())
mascara_nulos_hourly = df['hourlyrate($)'].isnull()

df.loc[mascara_nulos_hourly, 'hourlyrate($)'] = df['dailyrate($)'] / 8

print("Valores nulos DESPUÉS de la corrección de hourlyrate:", df['hourlyrate($)'].isnull().sum())
print("✅ Columna 'hourlyrate' corregida.")

Valores nulos ANTES de la corrección de hourlyrate: 1267
Valores nulos DESPUÉS de la corrección de hourlyrate: 0
✅ Columna 'hourlyrate' corregida.


In [52]:
#LIMPIEZA MONTHLYINCOME

print("\nValores nulos ANTES de la corrección de monthlyincome:", df['monthlyincome($)'].isnull().sum())
mascara_nulos_monthly = df['monthlyincome($)'].isnull()

mascara_calculo_monthly = mascara_nulos_monthly & df['salary($)'].notnull()

df.loc[mascara_calculo_monthly, 'monthlyincome($)'] = df['salary($)'] / 12

print("Valores nulos DESPUÉS de la corrección de monthlyincome:", df['monthlyincome($)'].isnull().sum())
print("✅ Columna 'monthlyincome' corregida (parcialmente, si hay nulos en salary).")


Valores nulos ANTES de la corrección de monthlyincome: 489
Valores nulos DESPUÉS de la corrección de monthlyincome: 0
✅ Columna 'monthlyincome' corregida (parcialmente, si hay nulos en salary).


In [53]:
#LIMPIEZA SALARY

print("\nValores nulos ANTES de la corrección de salary:", df['salary($)'].isnull().sum())
mascara_nulos_salary = df['salary($)'].isnull()

mascara_calculo_salary = mascara_nulos_salary & df['monthlyincome($)'].notnull()

df.loc[mascara_calculo_salary, 'salary'] = df['monthlyincome($)'] * 12

print("Valores nulos DESPUÉS de la corrección de salary:", df['salary($)'].isnull().sum())
print("✅ Columna 'salary' corregida (parcialmente, si quedan nulos en monthlyincome).")


Valores nulos ANTES de la corrección de salary: 285
Valores nulos DESPUÉS de la corrección de salary: 285
✅ Columna 'salary' corregida (parcialmente, si quedan nulos en monthlyincome).


In [54]:
#SEGUNDA LIMPIEZA DE SALARY

print("Nulos en salary ANTES de la doble pasada:", df['salary($)'].isnull().sum())

mascara_nulos_salary_final = df['salary($)'].isnull()

df.loc[mascara_nulos_salary_final, 'salary($)'] = df['monthlyincome($)'] * 12

print("Nulos en salary DESPUÉS de la doble pasada:", df['salary($)'].isnull().sum())

if df['salary($)'].isnull().sum() == 0:
    print("🎉 ¡La columna 'salary' ha sido corregida por completo!")
else:
    print("⚠️ Aún quedan nulos en 'salary'. Esto sugiere que monthlyincome tenía nulos irresolubles.")

Nulos en salary ANTES de la doble pasada: 285
Nulos en salary DESPUÉS de la doble pasada: 0
🎉 ¡La columna 'salary' ha sido corregida por completo!


In [55]:
df[['dailyrate($)', 'hourlyrate($)', 'monthlyincome($)', 'salary($)']].isna().sum()

dailyrate($)        0
hourlyrate($)       0
monthlyincome($)    0
salary($)           0
dtype: int64

In [56]:
# LIMPIEZA WORKLIFEBALANCE

print("Valores nulos ANTES de la corrección de worklifebalance:", df['worklifebalance'].isna().sum())

moda = df['worklifebalance'].mode()[0]
print("Valor de la moda:", moda)

moda = df['worklifebalance'].mode()[0]
df['worklifebalance'] = df['worklifebalance'].fillna(moda)

print("Valores nulos DESPUÉS de la corrección de worklifebalance:", df['worklifebalance'].isna().sum())


Valores nulos ANTES de la corrección de worklifebalance: 114
Valor de la moda: 3
Valores nulos DESPUÉS de la corrección de worklifebalance: 0
